# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [20]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sueldb WITH REPLICATION = 
                    {
                        'class':'SimpleStrategy','replication_factor': 1
                    }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [21]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sueldb')
except Exception as e:
    print(e)

### Creating queries to ask the three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [22]:
# Creating table to meet the requirements of QUERY #1
# We have used primary key because it is difficult to perform ad-hoc queries and No SQL DB has a 
# rule of one query-one table that why a primary key and clustering column method was seleted
# sessionId was selected as primary key along with itemInSession as clustering column because sessionId unique per user and each user will have 
# n items in his/her list.
try:
    query = "CREATE TABLE IF NOT EXISTS songInfo_by_sessionId"
    query = query + "(sessionId int, itemInSession int, artist text, gender text, song text, length float, userId int, PRIMARY KEY(sessionId, itemInSession))"
    
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
# Part of code to set up a csv file and INSERT Data to our table songInfo_by_sessionId using INSERT statement while iterating throught the csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO songInfo_by_sessionId(sessionId, itemInSession, artist, gender, song, length, userId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]), line[0], line[2], line[9], float(line[5]),  int(line[10])))

In [24]:
# QUERY 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM songInfo_by_sessionId WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(query)

# Varifying whether data inserted to the table is correct
for row in rows:
    print('The artist name is \'{}\', song name is \'{}\' and song\'s is \'{}\''.format(row.artist, row.song, row.length))

The artist name is 'Faithless', song name is 'Music Matters (Mark Knight Dub)' and song's is '495.30731201171875'


In [25]:
# Creating table to meet the requirements of QUERY #2

# Primary key was selected because it is difficult to perform ad-hoc queries and No SQL DB has a rule of one 
# query-one table that why a composite primary key and 
# clustering column method was seleted
# userId+sessionId was selected as composite key along with itemInSession as clustering column because userId + sessionId unique per user and each user will have 
# n items in his/her list.

# Part of code to set up a csv file and INSERT Data to our table songInfo_by_userId using INSERT statement while iterating throught the csv

query = "CREATE TABLE IF NOT EXISTS songInfo_by_userId"
query = query + "(userId int, sessionId int, itemInSession int, artist text, firstName text, lastName text, song text, PRIMARY KEY((userId,sessionId), itemInSession))"
                
session.execute(query)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO songInfo_by_userId(userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9] ))

In [26]:
# QUERY 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "SELECT artist, song, firstName, lastName FROM songInfo_by_userId WHERE userId=10 AND sessionId=182"
rows = session.execute(query)

# Varifying whether data inserted to the table is correct
for row in rows:
    print('The artist is \'{}\', the song is \'{}\', and User\'s name is \'{} {}\''.format(row.artist, row.song, row.firstname, row.lastname))

The artist is 'Down To The Bone', the song is 'Keep On Keepin' On', and User's name is 'Sylvie Cruz'
The artist is 'Three Drives', the song is 'Greece 2000', and User's name is 'Sylvie Cruz'
The artist is 'Sebastien Tellier', the song is 'Kilometer', and User's name is 'Sylvie Cruz'
The artist is 'Lonnie Gordon', the song is 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', and User's name is 'Sylvie Cruz'


In [27]:
# Creating table to meet the requirements of QUERY #3

# Part of code to set up a csv file and INSERT Data to our table songInfo_by_userId using INSERT statement while iterating throught the csv
query = "create table if not exists userDetails_by_song"
query += " (  song text, userId int, firstName text, lastName text, primary key(  song, userId))"

# song was selected as primary key as Apache will create a separate node based on for each song and each song will have a collection of userId that played
# the songs
session.execute(query)

In [28]:
# Part of code to set up a csv file and INSERT Data to our table songInfo_by_userId using INSERT statement while iterating throught the csv

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO userDetails_by_song( song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (  line[9],int(line[10]), line[1], line[4]))

In [29]:
# QUERY #3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT firstName, lastName from userDetails_by_song where song = 'All Hands Against His Own'"
rows = session.execute(query)

# Varifying whether data inserted to the table is correct
for row in rows:
    print('The user name is \'{} {}\''.format(row.firstname, row.lastname))

The user name is 'Jacqueline Lynch'
The user name is 'Tegan Levine'
The user name is 'Sara Johnson'


### Drop the tables before closing out the sessions

In [30]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE songInfo_by_sessionId")
session.execute("DROP TABLE songInfo_by_userId")
session.execute("DROP TABLE userDetails_by_song")

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()